In [1]:
import gdown

url = "https://drive.google.com/uc?id=1owa_NIBnj-Q5uwDyM2xBJeZPipjmkJsH"
output = "artificial_intelligence.json"

gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1owa_NIBnj-Q5uwDyM2xBJeZPipjmkJsH
To: /home/khuyen/Data-science/visualization/artificial_intelligence.json
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30.3M/30.3M [00:04<00:00, 6.21MB/s]


'artificial_intelligence.json'

In [2]:
import json

with open("artificial_intelligence.json") as f:
    output = json.load(f)
    
data = output["data"]

# View the keys of the first company
data[0].keys()

dict_keys(['twitterUri', 'nbActiveEmployeeEdges', 'type', 'allNames', 'revenue', 'yearlyRevenues', 'logo', 'id', 'stock', 'nbOrigins', 'sicClassification', 'foundingDate', 'image', 'images', 'wikipediaUri', 'irsEmployerIdentificationNumbers', 'diffbotUri', 'nbIncomingEdges', 'nbEmployeesMin', 'ipo', 'parentCompany', 'angellistUri', 'name', 'motto', 'nbEmployeesMax', 'totalInvestment', 'allOriginHashes', 'linkedInUri', 'naicsClassification', 'nbEmployees', 'githubUri', 'isDissolved', 'importance', 'origin', 'description', 'homepageUri', 'founders', 'ceo', 'investments', 'blogUri', 'descriptors', 'isNonProfit', 'origins', 'isPublic', 'categories', 'crawlTimestamp', 'nbUniqueInvestors', 'facebookUri', 'secCentralIndexKeys', 'summary', 'types', 'boardMembers', 'allUris', 'nbLocations', 'crunchbaseUri', 'industries', 'allDescriptions', 'location', 'locations', 'subsidiaries'])

In [4]:
from datetime import datetime
import pandas as pd 

def get_founding_year(company: dict):
    date_str = company.get("foundingDate", {}).get("str")
    if date_str:
        return date_str.split("-")[0][1:]
    return date_str
    
df = pd.DataFrame(
    {
        "company": [company.get("name") for company in data],
        "revenue": [company.get("revenue", {}).get("value") for company in data],
        "nbEmployees": [company.get("nbEmployees", None) for company in data],
        "founding_year": [get_founding_year(company) for company in data],
        "locations": [
            company.get("location", {}).get("country", {}).get("name")
            for company in data
        ],
        "isPublic": [company.get("isPublic") for company in data],
        "industries": [company.get("industries") for company in data],
    }
)

In [5]:
df = df[df.notnull().all(axis=1)]

df["founding_year"] = df["founding_year"].astype(int)

# Get number of years from now
df["year_from_now"] = df["founding_year"].apply(lambda row: datetime.now().year - row)

In [6]:
df.to_csv("artificial_intelligence.csv", index=False)